## set environment variables

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
INDEX_NAME = os.getenv("INDEX_NAME")

## libraries

In [2]:
from langchain_text_splitters import CharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from pinecone import Pinecone, ServerlessSpec
from langchain_pinecone import PineconeVectorStore

/home/tariq/Smart-Street/.venv/lib/python3.11/site-packages/pinecone/data/index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


## load the text data

In [6]:
dir = "violations_descriptsion_arabic"

texts = []

for text in os.listdir(dir):
    with open(f"{dir}/{text}", "r") as t:
        texts.append(t.read())

In [7]:
len(texts)

74

In [14]:
v = texts[3]
print(v)

20:45:03 تم رصد مخالفة مرورية في الوقت 
2024-09-22 في التاريخ الموافق 
24.765344, 46.645444 احداثيات المخالفة 
overtaking  نوع المخالفة 
nan رقم اللوحه
car نوع المركبة
اسم الطريق طريق الملك فهد الفرعي 
54رقم المخالفة


## splitting the texts into chunks

In [15]:
text_splitter = CharacterTextSplitter(
    separator="\n\n",
    chunk_size=30,
    chunk_overlap=3,
    length_function=len,
    is_separator_regex=False,
)

In [16]:
text_chunks = text_splitter.create_documents(texts)
print(f"number of chunks: {len(text_chunks)}")

number of chunks: 74


In [19]:
print(text_chunks[2])

page_content='20:44:53 تم رصد مخالفة مرورية في الوقت 
2024-09-22 في التاريخ الموافق 
24.6877, 46.7219 احداثيات المخالفة 
overtaking  نوع المخالفة 
nan رقم اللوحه
car نوع المركبة
اسم الطريق طريق المطار 
41رقم المخالفة'


### Embeddings and Storing in Pinecone

In [20]:
# Create Index
pc = Pinecone(api_key=PINECONE_API_KEY)

if 'violations-data-10' not in pc.list_indexes(): 
    pc.create_index(
        name='violations-data-11',
        dimension=1536,
        metric="cosine",
        spec=ServerlessSpec(
            cloud='aws',
            region='us-east-1'
        )
    )

index = pc.Index(INDEX_NAME)


In [21]:
embeddings = OpenAIEmbeddings(api_key=OPENAI_API_KEY, model="text-embedding-ada-002")
PineconeVectorStore.from_documents(text_chunks, embeddings, index_name='violations-data-11')